In [1]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [2]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 8, 3, padding=1)       #input: 28, Output: 28
        self.bn1 = nn.BatchNorm2d(8)
        self.drop = nn.Dropout(0.25)
        
        self.conv2 = nn.Conv2d(8, 16, 3, padding=1)      #input: 28, Output: 28
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)                  #input: 28, Output: 14
        self.drop = nn.Dropout(0.25)
        
        self.conv1x1_a = nn.Conv2d(16, 8, 1)
        
        self.conv3 = nn.Conv2d(8, 12, 3, padding=1)      #input: 14, Output: 14
        self.bn3 = nn.BatchNorm2d(12)
        self.drop = nn.Dropout(0.25)
        
        self.conv4 = nn.Conv2d(12, 16, 3, padding=1)     #input: 14, Output: 14
        self.bn4 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2)                  #input: 14, Output:  7
        self.drop = nn.Dropout(0.25)
        
        self.conv1x1_b = nn.Conv2d(16, 8, 1)
        
        self.conv5 = nn.Conv2d(8, 12, 3)                 #input:  7, Output:  5
        self.bn5 = nn.BatchNorm2d(12)
        self.drop = nn.Dropout(0.25)
        
        self.conv6 = nn.Conv2d(12, 16, 3)                #input:  5, Output:  3
        self.bn6 = nn.BatchNorm2d(16)
        
        self.conv7 = nn.Conv2d(16, 10, 3)                #input:  3, Output:  1

    def forward(self, x):
        x = self.pool1(self.bn2(F.relu(self.conv2(self.bn1(F.relu(self.conv1(x)))))))
        x = self.drop(x)
        x = self.conv1x1_a(x)
        x = self.pool2(self.bn4(F.relu(self.conv4(self.bn3(F.relu(self.conv3(x)))))))
        x = self.drop(x)
        x = self.conv1x1_b(x)
        x = self.bn6(F.relu(self.conv6(self.bn5(F.relu(self.conv5(x))))))
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=1)

In [3]:
# !pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
       BatchNorm2d-2            [-1, 8, 28, 28]              16
            Conv2d-3           [-1, 16, 28, 28]           1,168
       BatchNorm2d-4           [-1, 16, 28, 28]              32
         MaxPool2d-5           [-1, 16, 14, 14]               0
           Dropout-6           [-1, 16, 14, 14]               0
            Conv2d-7            [-1, 8, 14, 14]             136
            Conv2d-8           [-1, 12, 14, 14]             876
       BatchNorm2d-9           [-1, 12, 14, 14]              24
           Conv2d-10           [-1, 16, 14, 14]           1,744
      BatchNorm2d-11           [-1, 16, 14, 14]              32
        MaxPool2d-12             [-1, 16, 7, 7]               0
          Dropout-13             [-1, 16, 7, 7]               0
           Conv2d-14              [-1, 

In [4]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [5]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    correct = 0
    processed = 0
     
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()

        # Update pbar-tqdm

        pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
        correct += pred.eq(target.view_as(pred)).sum().item()
        processed += len(data)
        
        pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Accuracy={100*correct/processed:0.2f}')

        

def test(model, device, test_loader, epoch):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nEpoch : {} Test set: Average loss: {:.4f}, Accuracy: {}/{} ({:.2f}%)\n'.format(epoch,
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [6]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 21):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader, epoch)

Loss=0.1576959490776062 Batch_id=468 Accuracy=93.06: 100%|██████████| 469/469 [00:15<00:00, 30.35it/s]  
  0%|          | 0/469 [00:00<?, ?it/s]


Epoch : 1 Test set: Average loss: 0.0569, Accuracy: 9804/10000 (98.04%)



Loss=0.10972488671541214 Batch_id=468 Accuracy=97.52: 100%|██████████| 469/469 [00:15<00:00, 31.14it/s] 
  0%|          | 0/469 [00:00<?, ?it/s]


Epoch : 2 Test set: Average loss: 0.0410, Accuracy: 9863/10000 (98.63%)



Loss=0.05251230672001839 Batch_id=468 Accuracy=97.98: 100%|██████████| 469/469 [00:15<00:00, 30.33it/s] 
  0%|          | 0/469 [00:00<?, ?it/s]

...


Epoch : 18 Test set: Average loss: 0.0206, Accuracy: 9925/10000 (99.25%)



Loss=0.01726837083697319 Batch_id=468 Accuracy=99.25: 100%|██████████| 469/469 [00:15<00:00, 31.17it/s]  
  0%|          | 0/469 [00:00<?, ?it/s]


Epoch : 19 Test set: Average loss: 0.0186, Accuracy: 9942/10000 (99.42%)



Loss=0.03633946180343628 Batch_id=468 Accuracy=99.18: 100%|██████████| 469/469 [00:15<00:00, 30.62it/s]  



Epoch : 20 Test set: Average loss: 0.0196, Accuracy: 9940/10000 (99.40%)

